In [ ]:
!pip install pandas openpyxl scikit-learn tensorflow --quiet

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, r2_score

import tensorflow as tf
from tensorflow.keras import layers, Model


In [ ]:
from google.colab import files

uploaded = files.upload()  # 👉 choose Zizfondatabase1.xlsx

df = pd.read_excel("Zizfondatabase1.xlsx")

print("Shape:", df.shape)
print(df.columns.tolist())


Saving Zizfondatabase1.xlsx to Zizfondatabase1.xlsx
Shape: (438, 125)
['Country_ID', 'PROJECT_Name', 'YEAR', 'Project_Statu', 'Statu_ID', 'Sector', 'Evaluation', 'Regenoation', 'RevenueModle_type', 'VoiceOfAccountibilitry_StakeholderEngagment_', 'Macro_Enviromentimpact', 'Goverance_1', 'Regulority_Qualityassess', 'ControlofCorruption', 'BiddingTeam', 'PPP_Procuerment', 'Financial_capacity', 'Project_Financing', 'Financial_Viability', 'Partnership_collabration', 'Lifecycle_cost', 'Political_Risk', 'Sustaibablity', 'Risk_allocation', 'Contrcatural_Duties', 'Cost_efficency', 'Enviromrntal_Impact', 'Contrcact_complaince', 'StakholderSatisfaction', 'Outcome_spesification', 'Benchmarking', 'Servce_Quality', 'Time', 'Cost', 'Quality', 'Smart_innovation', 'Conssession_period', 'Sucess_Index', 'Sucess_Binary', 'PRE_1', 'ADJ_1', 'ZPR_1', 'SEP_1', 'PRE_2', 'ADJ_2', 'ZPR_2', 'SEP_2', 'PRE_3', 'ADJ_3', 'ZPR_3', 'SEP_3', 'PRE_4', 'ADJ_4', 'ZPR_4', 'SEP_4', 'PRE_5', 'ADJ_5', 'ZPR_5', 'SEP_5', 'PRE_6'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ---- Targets (heads) ----
target_cols = ["Evaluation", "Sucess_Index", "Contrcact_complaince", "Quality"]

# ---- Features (IVs) ----
feature_cols = [
    "Country_ID","YEAR","Project_Statu","Statu_ID","Sector",
    "Regenoation","RevenueModle_type",
    "VoiceOfAccountibilitry_StakeholderEngagment_",
    "Macro_Enviromentimpact","Goverance_1",
    "Regulority_Qualityassess","ControlofCorruption",
    "BiddingTeam","PPP_Procuerment","Financial_capacity",
    "Project_Financing","Financial_Viability",
    "Partnership_collabration","Lifecycle_cost",
    "Political_Risk","Sustaibablity","Risk_allocation",
    "Contrcatural_Duties","Cost_efficency","Enviromrntal_Impact",
    "StakholderSatisfaction","Outcome_spesification","Benchmarking",
    "Servce_Quality","Time","Cost","Smart_innovation",
    "Conssession_period"
]

# Keep only columns that actually exist
feature_cols = [c for c in feature_cols if c in df.columns]
cols_to_use = feature_cols + target_cols

# Drop rows with missing values in any of these columns
df_clean = df.dropna(subset=cols_to_use).copy()

print("Rows after cleaning:", df_clean.shape[0])

# ---- Build y (targets) ----
y_eval    = df_clean["Evaluation"].values
y_sindex  = df_clean["Sucess_Index"].values
y_comp    = df_clean["Contrcact_complaince"].values
y_quality = df_clean["Quality"].values

# ---- Build X as a DataFrame and one-hot encode categoricals ----
X_raw = df_clean[feature_cols]

# Convert all non-numeric columns into dummies
X_encoded = pd.get_dummies(X_raw, drop_first=True)

print("Encoded feature shape:", X_encoded.shape)
print("Example encoded columns:", X_encoded.columns[:20])

X = X_encoded.values

# ---- Train / test split ----
all_y = np.column_stack([y_eval, y_sindex, y_comp, y_quality])

X_train, X_test, y_train, y_test = train_test_split(
    X, all_y, test_size=0.2, random_state=42
)

y_eval_train, y_sindex_train, y_comp_train, y_qual_train = y_train.T
y_eval_test,  y_sindex_test,  y_comp_test,  y_qual_test  = y_test.T

# ---- Scale numeric features ----
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)


Rows after cleaning: 228
Encoded feature shape: (228, 94)
Example encoded columns: Index(['YEAR', 'Statu_ID', 'Sector', 'Regenoation', 'RevenueModle_type',
       'VoiceOfAccountibilitry_StakeholderEngagment_',
       'Macro_Enviromentimpact', 'Goverance_1', 'Regulority_Qualityassess',
       'ControlofCorruption', 'BiddingTeam', 'PPP_Procuerment',
       'Financial_capacity', 'Project_Financing', 'Financial_Viability',
       'Partnership_collabration', 'Lifecycle_cost', 'Political_Risk',
       'Sustaibablity', 'Risk_allocation'],
      dtype='object')
X_train_scaled shape: (182, 94)
X_test_scaled shape: (46, 94)


In [ ]:
input_dim = X_train_scaled.shape[1]

inputs = layers.Input(shape=(input_dim,), name="project_features")

# Shared backbone
x = layers.Dense(128, activation="relu")(inputs)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.3)(x)

# ---- Head 1: Evaluation (classification 0/1) ----
h_eval = layers.Dense(32, activation="relu")(x)
out_eval = layers.Dense(1, activation="sigmoid", name="eval_output")(h_eval)

# ---- Head 2: Success Index (regression) ----
h_sindex = layers.Dense(32, activation="relu")(x)
out_sindex = layers.Dense(1, activation="linear", name="sindex_output")(h_sindex)

# ---- Head 3: Contract compliance (regression) ----
h_comp = layers.Dense(32, activation="relu")(x)
out_comp = layers.Dense(1, activation="linear", name="comp_output")(h_comp)

# ---- Head 4: Quality (regression or score) ----
h_qual = layers.Dense(32, activation="relu")(x)
out_qual = layers.Dense(1, activation="linear", name="qual_output")(h_qual)

model = Model(
    inputs=inputs,
    outputs=[out_eval, out_sindex, out_comp, out_qual],
    name="PPP_MultiHead_DNN"
)

model.summary()


Model: "PPP_MultiHead_DNN"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ project_features    │ (None, 94)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     12,160 │ project_features… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      2,080 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 32)        │      2,080 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │      2,080 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ eval_output (Dense) │ (None, 1)         │         33 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sindex_output       │ (None, 1)         │         33 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ comp_output (Dense) │ (None, 1)         │         33 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ qual_output (Dense) │ (None, 1)         │         33 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 28,868 (112.77 KB)

 Trainable params: 28,868 (112.77 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss={
        "eval_output": "binary_crossentropy",  # Evaluation = 0/1
        "sindex_output": "mse",
        "comp_output": "mse",
        "qual_output": "mse",
    },
    loss_weights={
        "eval_output": 1.0,
        "sindex_output": 1.0,
        "comp_output": 0.8,
        "qual_output": 0.8,
    },
    metrics={
        "eval_output": ["accuracy"],  # we mainly track accuracy for this head
    }
)


In [ ]:
history = model.fit(
    X_train_scaled,
    {
        "eval_output": y_eval_train,
        "sindex_output": y_sindex_train,
        "comp_output": y_comp_train,
        "qual_output": y_qual_train,
    },
    validation_data=(
        X_test_scaled,
        {
            "eval_output": y_eval_test,
            "sindex_output": y_sindex_test,
            "comp_output": y_comp_test,
            "qual_output": y_qual_test,
        }
    ),
    epochs=100,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=10,
            restore_best_weights=True
        )
    ],
    verbose=1
)


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - comp_output_loss: 7398.8711 - eval_output_accuracy: 0.4705 - eval_output_loss: 0.7547 - loss: 5972.8677 - qual_output_loss: 0.5779 - sindex_output_loss: 52.0071 - val_comp_output_loss: 7391.9966 - val_eval_output_accuracy: 0.6522 - val_eval_output_loss: 0.6369 - val_loss: 5809.2964 - val_qual_output_loss: 0.3664 - val_sindex_output_loss: 45.9916
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - comp_output_loss: 7167.1851 - eval_output_accuracy: 0.6509 - eval_output_loss: 0.6369 - loss: 5775.0391 - qual_output_loss: 0.6508 - sindex_output_loss: 40.8019 - val_comp_output_loss: 7227.6621 - val_eval_output_accuracy: 0.6739 - val_eval_output_loss: 0.6178 - val_loss: 5669.6523 - val_qual_output_loss: 0.3558 - val_sindex_output_loss: 36.8732
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - comp_output_loss: 7079.5210 - eval_output_accuracy: 0.7498 - eval_output_loss: 0.5845 - loss: 5698.3774 - qual_output_loss: 0.5090 - sindex_output_l

In [ ]:
from sklearn.metrics import accuracy_score, r2_score

# Evaluate with Keras
results = model.evaluate(
    X_test_scaled,
    {
        "eval_output": y_eval_test,
        "sindex_output": y_sindex_test,
        "comp_output": y_comp_test,
        "qual_output": y_qual_test,
    },
    verbose=0
)
print("Raw Keras results:", results)

# Predictions
y_eval_pred_prob, y_sindex_pred, y_comp_pred, y_qual_pred = model.predict(X_test_scaled)

# Classification performance for Evaluation
y_eval_pred_class = (y_eval_pred_prob.flatten() >= 0.5).astype(int)
eval_acc = accuracy_score(y_eval_test, y_eval_pred_class)
print("Evaluation head accuracy:", eval_acc)

# R² for regression heads
r2_sindex = r2_score(y_sindex_test, y_sindex_pred)
r2_comp   = r2_score(y_comp_test, y_comp_pred)
r2_qual   = r2_score(y_qual_test, y_qual_pred)

print("R² – Success Index:", r2_sindex)
print("R² – Contract compliance:", r2_comp)
print("R² – Quality:", r2_qual)


Raw Keras results: [175.12091064453125, 0.5043803453445435, 6.425505638122559, 178.5874786376953, 0.24239420890808105, 0.717391312122345]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Evaluation head accuracy: 0.717391304347826
R² – Success Index: -2.917900790893448
R² – Contract compliance: 0.08453360121378029
R² – Quality: -0.12696716876295744


In [ ]:
df_clean = df.dropna(subset=cols_to_use).copy()


In [ ]:
# Scale contract compliance from 1–100 to 0–1
df_clean["Contrcact_complaince_scaled"] = df_clean["Contrcact_complaince"] / 100.0


In [ ]:
y_eval    = df_clean["Evaluation"].values              # 0/1
y_comp    = df_clean["Contrcact_complaince_scaled"].values  # 0–1 continuous
y_quality = df_clean["Quality"].values                 # 0/1


In [ ]:
all_y = np.column_stack([y_eval, y_comp, y_quality])

X_train, X_test, y_train, y_test = train_test_split(
    X, all_y, test_size=0.2, random_state=42
)

y_eval_train, y_comp_train, y_qual_train = y_train.T
y_eval_test,  y_comp_test,  y_qual_test  = y_test.T


In [ ]:
input_dim = X_train_scaled.shape[1]

inputs = layers.Input(shape=(input_dim,), name="project_features")

# Shared backbone
x = layers.Dense(128, activation="relu")(inputs)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.3)(x)

# ---- Head 1: Evaluation (classification 0/1) ----
h_eval = layers.Dense(32, activation="relu")(x)
out_eval = layers.Dense(1, activation="sigmoid", name="eval_output")(h_eval)

# ---- Head 2: Contract compliance (regression, 0–1) ----
h_comp = layers.Dense(32, activation="relu")(x)
out_comp = layers.Dense(1, activation="linear", name="comp_output")(h_comp)

# ---- Head 3: Quality (classification 0/1) ----
h_qual = layers.Dense(32, activation="relu")(x)
out_qual = layers.Dense(1, activation="sigmoid", name="qual_output")(h_qual)

model = Model(
    inputs=inputs,
    outputs=[out_eval, out_comp, out_qual],
    name="PPP_MultiHead_DNN_3heads"
)

model.summary()


Model: "PPP_MultiHead_DNN_3heads"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ project_features    │ (None, 94)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     12,160 │ project_features… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │      8,256 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 64)        │          0 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │      2,080 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 32)        │      2,080 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ eval_output (Dense) │ (None, 1)         │         33 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ comp_output (Dense) │ (None, 1)         │         33 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ qual_output (Dense) │ (None, 1)         │         33 │ dense_10[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 26,755 (104.51 KB)

 Trainable params: 26,755 (104.51 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss={
        "eval_output": "binary_crossentropy",  # Evaluation = 0/1
        "comp_output": "mse",                  # Compliance = continuous 0–1
        "qual_output": "binary_crossentropy",  # Quality = 0/1
    },
    loss_weights={
        "eval_output": 1.0,
        "comp_output": 1.0,
        "qual_output": 1.0,
    },
    metrics={
        "eval_output": ["accuracy"],
        "qual_output": ["accuracy"],
    }
)


In [ ]:
history = model.fit(
    X_train_scaled,
    {
        "eval_output": y_eval_train,
        "comp_output": y_comp_train,
        "qual_output": y_qual_train,
    },
    validation_data=(
        X_test_scaled,
        {
            "eval_output": y_eval_test,
            "comp_output": y_comp_test,
            "qual_output": y_qual_test,
        }
    ),
    epochs=100,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=10,
            restore_best_weights=True
        )
    ],
    verbose=1
)


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - comp_output_loss: 1.2738 - eval_output_accuracy: 0.4998 - eval_output_loss: 0.7032 - loss: 2.7248 - qual_output_accuracy: 0.5106 - qual_output_loss: 0.7398 - val_comp_output_loss: 0.1722 - val_eval_output_accuracy: 0.6957 - val_eval_output_loss: 0.6061 - val_loss: 1.4487 - val_qual_output_accuracy: 0.6304 - val_qual_output_loss: 0.6708
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - comp_output_loss: 0.4419 - eval_output_accuracy: 0.7119 - eval_output_loss: 0.6180 - loss: 1.7227 - qual_output_accuracy: 0.6848 - qual_output_loss: 0.6705 - val_comp_output_loss: 0.1410 - val_eval_output_accuracy: 0.6739 - val_eval_output_loss: 0.5710 - val_loss: 1.3514 - val_qual_output_accuracy: 0.6522 - val_qual_output_loss: 0.6322
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - comp_output_loss: 0.2914 - eval_output_accuracy: 0.7400 - eval_output_loss: 0.6096 - loss: 1.5095 - qual_output_accuracy: 0.7257 - qual_output_loss: 0.6062 - val_comp_o

In [ ]:
from sklearn.metrics import accuracy_score, r2_score

results = model.evaluate(
    X_test_scaled,
    {
        "eval_output": y_eval_test,
        "comp_output": y_comp_test,
        "qual_output": y_qual_test,
    },
    verbose=0
)
print("Raw Keras results:", results)

# Predictions
y_eval_pred_prob, y_comp_pred, y_qual_pred_prob = model.predict(X_test_scaled)

# Evaluation accuracy
y_eval_pred_class = (y_eval_pred_prob.flatten() >= 0.5).astype(int)
eval_acc = accuracy_score(y_eval_test, y_eval_pred_class)
print("Evaluation head accuracy:", eval_acc)

# Quality accuracy
y_qual_pred_class = (y_qual_pred_prob.flatten() >= 0.5).astype(int)
qual_acc = accuracy_score(y_qual_test, y_qual_pred_class)
print("Quality head accuracy:", qual_acc)

# Compliance R²
r2_comp = r2_score(y_comp_test, y_comp_pred)
print("R² – Contract compliance (scaled 0–1):", r2_comp)


Raw Keras results: [0.5835477113723755, 0.20055432617664337, 0.0460585355758667, 0.29758143424987793, 0.9347826242446899, 0.9130434989929199]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Evaluation head accuracy: 0.9347826086956522
Quality head accuracy: 0.9130434782608695
R² – Contract compliance (scaled 0–1): -0.954307597860429


In [ ]:
Input layer: 6–10 variables
Hidden layer 1: 16 neurons (ReLU)
Hidden layer 2: 8 neurons (ReLU)
Output layer: 1 neuron (Sigmoid)


SyntaxError: invalid character '–' (U+2013) (ipython-input-2386111220.py, line 1)